<a href="https://colab.research.google.com/github/PhoenixAlpha23/Fine-tune-embeddings/blob/main/fine-tune_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing dependencies
tesseract-ocr ,datasets, sentence transformers,nltk and unstructured for documents.

In [2]:
!sudo apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.


In [3]:
!pip install poppler-utils tesseract-ocr
!pip install datasets sentence-transformers google-generativeai
!pip install -q --user --upgrade pillow
!pip install -q unstructured["all-docs"] pi_heif
!pip install -q --upgrade unstructured
!pip install --upgrade nltk

  Using cached poppler_utils-0.1.0-py3-none-any.whl.metadata (883 bytes)
  Using cached tesseract-ocr-0.0.1.tar.gz (33 kB)
  Preparing metadata (setup.py) ... done
Using cached poppler_utils-0.1.0-py3-none-any.whl (9.2 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tesseract-ocr
  Running setup.py clean for tesseract-ocr
Failed to build tesseract-ocr
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tesseract-ocr)


In [6]:
!apt-get update
!apt-get install poppler-utils -y

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (132 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package list

In [8]:
!pip install pdfminer.six
!apt-get install poppler-utils -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.


In [5]:
import os
import sys
# Find the directory where poppler-utils is installed
poppler_path = os.path.join(sys.prefix, "bin")

# Add the directory to the system PATH
os.environ["PATH"] += os.pathsep + poppler_path

from IPython import get_ipython
from IPython.display import display

## PDF parsing and Text extraction

In [9]:
import nltk
import os
from unstructured.partition.pdf import partition_pdf
from collections import Counter
import warnings
from pdfminer.pdfparser import PDFSyntaxError

# Download NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')


def process_pdfs_in_folder(folder_path):
    total_text = []  # To accumulate the text from all PDFs

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")

    #list of all PDF files in the folder
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

    if not pdf_files:
        print("No PDF files found in the folder.")
        return ""

    for pdf_file in pdf_files:
        pdf_path = os.path.join(folder_path, pdf_file)
        print(f"Processing: {pdf_path}")
        try:
            #partition logic
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", category=UserWarning)  # Suppress warnings
                elements = partition_pdf(pdf_path, strategy="auto")
            # Display the types of elements
            display(Counter(type(element) for element in elements))
            # Join the elements to form text and add it to total_text list
            text = "\n\n".join([str(el) for el in elements])
            total_text.append(text)
        except PDFSyntaxError:
            print(f"Skipping {pdf_file}: Invalid or corrupted PDF file.")
        except Exception as e:
            print(f"Error processing {pdf_file}: {e}")

    # Return the total concatenated text
    return "\n\n".join(total_text)


folder_path = "data"
all_text = process_pdfs_in_folder(folder_path)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Processing: data/Schargesheet-1.pdf


Counter({unstructured.documents.elements.Title: 57,
         unstructured.documents.elements.Text: 9,
         unstructured.documents.elements.NarrativeText: 36,
         unstructured.documents.elements.ListItem: 4})

Processing: data/fNy5D2yT2Lgrpup9gNVMg3hSbRH6pVBrmcO6LNhI.pdf
Error processing fNy5D2yT2Lgrpup9gNVMg3hSbRH6pVBrmcO6LNhI.pdf: Unable to get page count.
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table

Processing: data/scs1-1.pdf


Counter({unstructured.documents.elements.Title: 152,
         unstructured.documents.elements.Text: 32,
         unstructured.documents.elements.ListItem: 11,
         unstructured.documents.elements.NarrativeText: 56})

## Self-defined text chunking

In [10]:
import nltk

nltk.download('punkt')

def nltk_based_splitter(text: str, chunk_size: int, overlap: int) -> list:
    """
    Splits the input text into chunks of a specified size, with optional overlap between chunks.

    Parameters:
    - text: The input text to be split.
    - chunk_size: The maximum size of each chunk (in terms of characters).
    - overlap: The number of overlapping characters between consecutive chunks.

    Returns:
    - A list of text chunks, with or without overlap.
    """

    from nltk.tokenize import sent_tokenize

    # Tokenize the input text into individual sentences
    sentences = sent_tokenize(text)

    chunks = []
    current_chunk = ""

    for sentence in sentences:
        # If the current chunk plus the next sentence doesn't exceed the chunk size, add the sentence to the chunk
        if len(current_chunk) + len(sentence) <= chunk_size:
            current_chunk += " " + sentence
        else:
            # Otherwise, add the current chunk to the list of chunks and start a new chunk with the current sentence
            chunks.append(current_chunk.strip())  # Strip to remove leading spaces
            current_chunk = sentence

    # After the loop, if there is any leftover text in the current chunk, add it to the list of chunks
    if current_chunk:
        chunks.append(current_chunk.strip())

    # Handle overlap if it's specified (overlap > 0)
    if overlap > 0:
        overlapping_chunks = []
        for i in range(len(chunks)):
            if i > 0:
                # Calculate the start index for overlap from the previous chunk
                start_overlap = max(0, len(chunks[i-1]) - overlap)
                # Combine the overlapping portion of the previous chunk with the current chunk
                chunk_with_overlap = chunks[i-1][start_overlap:] + " " + chunks[i]
                # Append the combined chunk, making sure it's not longer than chunk_size
                overlapping_chunks.append(chunk_with_overlap[:chunk_size])
            else:
                # For the first chunk, there's no previous chunk to overlap with
                overlapping_chunks.append(chunks[i][:chunk_size])

        return overlapping_chunks  # Return the list of chunks with overlap

    # If overlap is 0, return the non-overlapping chunks
    return chunks

chunks = nltk_based_splitter(text=all_text,
                                  chunk_size=2048,
                                  overlap=0)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Dataset generation
To create Question and Answer pairs from our dataset,using gemini API.

In [11]:
import google.generativeai as genai
import pandas as pd

# Replace with your valid Google API key
GOOGLE_API_KEY = "YOUR_API_KEY"

# Prompt generator with an explicit request for structured output
def prompt(text_chunk):
    return f"""
    Based on the following text, generate one Question and its corresponding Answer.
    Please format the output as follows:
    Question: [Your question]
    Answer: [Your answer]

    Text: {text_chunk}
    """
# Function to interact with Google's Gemini and return a QA pair
def generate_with_gemini(text_chunk:str, temperature:float, model_name:str):
    genai.configure(api_key=GOOGLE_API_KEY)
    generation_config = {"temperature": temperature}

    # Initialize the generative model
    gen_model = genai.GenerativeModel(model_name, generation_config=generation_config)

    # Generate response based on the prompt
    response = gen_model.generate_content(prompt(text_chunk))

    # Extract question and answer from response using keyword
    try:
        question, answer = response.text.split("Answer:", 1)
        question = question.replace("Question:", "").strip()
        answer = answer.strip()
    except ValueError:
        question, answer = "N/A", "N/A"  # Handle unexpected format in response

    return question, answer

process_text_chunks function is used to create QnA pairs for each text chunkusing gemini model.

In [12]:
def process_text_chunks(text_chunks:list, temperature:int, model_name=str):
    """
    Processes a list of text chunks to generate questions and answers using a specified model.

    Parameters:
    - text_chunks: A list of text chunks to process.
    - temperature: The sampling temperature to control randomness in the generated outputs.
    - model_name: The name of the model to use for generating questions and answers.

    Returns:
    - A Pandas DataFrame containing the text chunks, questions, and answers.
    """
    results = []

    # Iterate through each text chunk
    for chunk in text_chunks:
        question, answer = generate_with_gemini(chunk, temperature, model_name)
        results.append({"Text Chunk": chunk, "Question": question, "Answer": answer})

    # Convert results into a Pandas DataFrame
    df = pd.DataFrame(results)
    return df
# Process the text chunks and get the DataFrame
df_results = process_text_chunks(text_chunks=chunks,
                                 temperature=0.7,
                                 model_name="gemini-1.5-flash")
df_results.to_csv("generated_qa_pairs.csv", index=False)

# Load the CSV file into a Hugging Face Dataset

In [13]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='generated_qa_pairs.csv')

def process_example(example, idx):
    return {
        "id": idx,  #unique ID based on the index
        "anchor": example["Question"],
        "positive": example["Answer"]
    }
dataset = dataset.map(process_example,
                      with_indices=True ,
                      remove_columns=["Text Chunk", "Question", "Answer"])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

I have used the BAAI/bge-base-en-v1.5 model, developed by BAAI (Beijing Academy of Artificial Intelligence) since it is a powerful text embedding model, it excels at various NLP tasks and has been shown to perform well on benchmarks like MTEB and C-MTEB.

The bge-base-en model is a good choice for applications with limited computational resources, such as this one:)

In [14]:
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import (
    InformationRetrievalEvaluator,
    SequentialEvaluator,
)
from sentence_transformers.util import cos_sim
from datasets import load_dataset, concatenate_datasets
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss


model_id = "BAAI/bge-base-en-v1.5"

# Load model
model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)

## **Matryoshka Fine-tuning:** Fine-tunes the BGE model using Matryoshka loss to enhance retrieval accuracy.

Why to you use 🪆 Matryoshka Embedding models?
Such variable-size embedding modelshelp in:

* Shortlisting and reranking: Rather than performing your downstream task (e.g., nearest neighbor search) on the full embeddings, you can shrink the embeddings to a smaller size and very efficiently "shortlist" your embeddings.

* Afterwards, you can process the remaining embeddings using their full dimensionality.

* *Trade-offs:* Matryoshka models will allow you to scale your embedding solutions to your desired storage cost, processing speed, and performance.

In [15]:
# Essential to set them from large to small, like the matryoshka dolls.
matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [23]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers

# define training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="bge-finetuned",                 # output directory and hugging face model ID
    num_train_epochs=5,                         # number of epochs
    per_device_train_batch_size=4,              # train batch size
    gradient_accumulation_steps=16,             # for a global batch size of 512
    per_device_eval_batch_size=16,              # evaluation batch size
    warmup_ratio=0.1,                           # warmup ratio
    learning_rate=2e-5,                         # learning rate
    lr_scheduler_type="cosine",                 # use constant learning rate scheduler
    optim="adamw_torch_fused",                  # use fused adamw optimizer
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="epoch",                      # evaluate after each epoch
    save_strategy="epoch",                      # save after each epoch
    logging_steps=10,                           # log every 10 steps
    save_total_limit=3,                         # save only the last 3 models
    load_best_model_at_end=True,                # load the best model when training ends
    metric_for_best_model="eval_dim_128_cosine_ndcg@10",  # Optimizing for the best ndcg@10 score for the 128 dimension
)

In [17]:
corpus = dict(
    zip(dataset['train']['id'],
        dataset['train']['positive'])
)  # Our corpus (cid => document)

queries = dict(
    zip(dataset['train']['id'],
        dataset['train']['anchor'])
)  # Our queries (qid => question)

# Create a mapping of relevant documents for each query
relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for q_id in queries:
    relevant_docs[q_id] = [q_id]

matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=corpus,
        relevant_docs=relevant_docs,
        name=f"dim_{dim}",
        truncate_dim=dim,  # Truncate the embeddings to a certain dimension
        score_functions={"cosine": cos_sim},
    )
    matryoshka_evaluators.append(ir_evaluator)
#Create evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

In [18]:
#Model Evaluation before training
results = evaluator(model)

for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print(f"{key}: {results[key]}")

dim_768_cosine_ndcg@10: 0.8454569391354841
dim_512_cosine_ndcg@10: 0.8454569391354841
dim_256_cosine_ndcg@10: 0.8454569391354841
dim_128_cosine_ndcg@10: 0.8345188373870667
dim_64_cosine_ndcg@10: 0.7941518299738394


In [25]:
from sentence_transformers import SentenceTransformerTrainer

# Split the dataset into train and eval sets
train_dataset = dataset['train']
eval_dataset = dataset['train']

trainer = SentenceTransformerTrainer(
    model=model, # our embedding model
    args=args,  # defined above
    train_dataset=train_dataset.select_columns(
        ["positive", "anchor"]
    ),
    eval_dataset=eval_dataset.select_columns(
        ["positive", "anchor"]
    ),
    loss=train_loss, # Matryoshka loss
    evaluator=evaluator, # Sequential Evaluator
)

In [26]:
# start training
trainer.train()
# save the best model
trainer.save_model()

dataset = dataset.select_columns(['anchor', 'positive', 'negative'])


Epoch,Training Loss,Validation Loss,Dim 768 Cosine Accuracy@1,Dim 768 Cosine Accuracy@3,Dim 768 Cosine Accuracy@5,Dim 768 Cosine Accuracy@10,Dim 768 Cosine Precision@1,Dim 768 Cosine Precision@3,Dim 768 Cosine Precision@5,Dim 768 Cosine Precision@10,Dim 768 Cosine Recall@1,Dim 768 Cosine Recall@3,Dim 768 Cosine Recall@5,Dim 768 Cosine Recall@10,Dim 768 Cosine Ndcg@10,Dim 768 Cosine Mrr@10,Dim 768 Cosine Map@100,Dim 512 Cosine Accuracy@1,Dim 512 Cosine Accuracy@3,Dim 512 Cosine Accuracy@5,Dim 512 Cosine Accuracy@10,Dim 512 Cosine Precision@1,Dim 512 Cosine Precision@3,Dim 512 Cosine Precision@5,Dim 512 Cosine Precision@10,Dim 512 Cosine Recall@1,Dim 512 Cosine Recall@3,Dim 512 Cosine Recall@5,Dim 512 Cosine Recall@10,Dim 512 Cosine Ndcg@10,Dim 512 Cosine Mrr@10,Dim 512 Cosine Map@100,Dim 256 Cosine Accuracy@1,Dim 256 Cosine Accuracy@3,Dim 256 Cosine Accuracy@5,Dim 256 Cosine Accuracy@10,Dim 256 Cosine Precision@1,Dim 256 Cosine Precision@3,Dim 256 Cosine Precision@5,Dim 256 Cosine Precision@10,Dim 256 Cosine Recall@1,Dim 256 Cosine Recall@3,Dim 256 Cosine Recall@5,Dim 256 Cosine Recall@10,Dim 256 Cosine Ndcg@10,Dim 256 Cosine Mrr@10,Dim 256 Cosine Map@100,Dim 128 Cosine Accuracy@1,Dim 128 Cosine Accuracy@3,Dim 128 Cosine Accuracy@5,Dim 128 Cosine Accuracy@10,Dim 128 Cosine Precision@1,Dim 128 Cosine Precision@3,Dim 128 Cosine Precision@5,Dim 128 Cosine Precision@10,Dim 128 Cosine Recall@1,Dim 128 Cosine Recall@3,Dim 128 Cosine Recall@5,Dim 128 Cosine Recall@10,Dim 128 Cosine Ndcg@10,Dim 128 Cosine Mrr@10,Dim 128 Cosine Map@100,Dim 64 Cosine Accuracy@1,Dim 64 Cosine Accuracy@3,Dim 64 Cosine Accuracy@5,Dim 64 Cosine Accuracy@10,Dim 64 Cosine Precision@1,Dim 64 Cosine Precision@3,Dim 64 Cosine Precision@5,Dim 64 Cosine Precision@10,Dim 64 Cosine Recall@1,Dim 64 Cosine Recall@3,Dim 64 Cosine Recall@5,Dim 64 Cosine Recall@10,Dim 64 Cosine Ndcg@10,Dim 64 Cosine Mrr@10,Dim 64 Cosine Map@100,Sequential Score
1,No log,5.759125,0.666667,0.933333,0.933333,1.000000,0.666667,0.311111,0.186667,0.100000,0.666667,0.933333,0.933333,1.000000,0.845457,0.795556,0.795556,0.666667,0.933333,0.933333,1.000000,0.666667,0.311111,0.186667,0.100000,0.666667,0.933333,0.933333,1.000000,0.845457,0.795556,0.795556,0.666667,0.933333,0.933333,1.000000,0.666667,0.311111,0.186667,0.100000,0.666667,0.933333,0.933333,1.000000,0.845457,0.795556,0.795556,0.733333,0.866667,0.933333,0.933333,0.733333,0.288889,0.186667,0.093333,0.733333,0.866667,0.933333,0.933333,0.834519,0.802222,0.807350,0.600000,0.800000,0.933333,1.000000,0.600000,0.266667,0.186667,0.100000,0.600000,0.800000,0.933333,1.000000,0.794152,0.728889,0.728889,0.794152
2,No log,3.886112,0.666667,0.933333,0.933333,1.000000,0.666667,0.311111,0.186667,0.100000,0.666667,0.933333,0.933333,1.000000,0.849933,0.800000,0.800000,0.733333,0.933333,0.933333,1.000000,0.733333,0.311111,0.186667,0.100000,0.733333,0.933333,0.933333,1.000000,0.874538,0.833333,0.833333,0.800000,0.933333,0.933333,1.000000,0.800000,0.311111,0.186667,0.100000,0.800000,0.933333,0.933333,1.000000,0.899142,0.866667,0.866667,0.733333,0.933333,0.933333,0.933333,0.733333,0.311111,0.186667,0.093333,0.733333,0.933333,0.933333,0.933333,0.850791,0.822222,0.827778,0.733333,0.933333,0.933333,1.000000,0.733333,0.311111,0.186667,0.100000,0.733333,0.933333,0.933333,1.000000,0.874538,0.833333,0.833333,0.874538
3,No log,2.903320,0.733333,1.000000,1.000000,1.000000,0.733333,0.333333,0.200000,0.100000,0.733333,1.000000,1.000000,1.000000,0.884124,0.844444,0.844444,0.733333,1.000000,1.000000,1.000000,0.733333,0.333333,0.200000,0.100000,0.733333,1.000000,1.000000,1.000000,0.884124,0.844444,0.844444,0.800000,0.933333,1.000000,1.000000,0.800000,0.311111,0.200000,0.100000,0.800000,0.933333,1.000000,1.000000,0.904107,0.872222,0.872222,0.733333,0.933333,0.933333,1.000000,0.733333,0.311111,0.186667,0.100000,0.733333,0.933333,0.933333,1.000000,0.870062,0.828889,0.828889,0.733333,1.000000,1.000000,1.000000,0.733333,0.333333,0.200000,0.100000,0.733333,1.000000,1.000000,1.0000

In [27]:
from sentence_transformers import SentenceTransformer

fine_tuned_model = SentenceTransformer(
    args.output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)
# Evaluatation of model after fine-tuning
results = evaluator(fine_tuned_model)

#main score
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print(f"{key}: {results[key]}")

dim_768_cosine_ndcg@10: 0.884123967142861
dim_512_cosine_ndcg@10: 0.884123967142861
dim_256_cosine_ndcg@10: 0.9261859507142916
dim_128_cosine_ndcg@10: 0.8745377796167292
dim_64_cosine_ndcg@10: 0.9174573004761943


In [28]:
from sklearn.metrics import ndcg_score

#Required:
# - `queries`: A dictionary where keys are query IDs and values are query texts.
# - `corpus`: A dictionary where keys are document IDs and values are document texts.
# - `relevant_docs`: A dictionary where keys are query IDs and values are lists of relevant document IDs.

def calculate_ndcg(model, queries, corpus, relevant_docs, k=10):
    """
    Calculates the NDCG@k score for a given model, queries, corpus, and relevant documents.

    Args:
        model: The SentenceTransformer model.
        queries: A dictionary of queries (query_id => query_text).
        corpus: A dictionary of documents (doc_id => doc_text).
        relevant_docs: A dictionary of relevant documents for each query (query_id => [relevant_doc_ids]).
        k: The number of top results to consider for NDCG calculation.

    Returns:
        The average NDCG@k score across all queries.
    """

    ndcg_scores = []

    for query_id, query_text in queries.items():
        # Encode the query
        query_embedding = model.encode(query_text)

        # Encode all documents in the corpus
        document_embeddings = model.encode(list(corpus.values()))

        # Calculate cosine similarities between query and documents
        similarities = cos_sim(query_embedding, document_embeddings).cpu().numpy()[0]

        # Get the indices of the top k most similar documents
        top_k_indices = similarities.argsort()[-k:][::-1]

        # Get the IDs of the top k documents
        retrieved_doc_ids = [list(corpus.keys())[i] for i in top_k_indices]

        # Create a binary relevance vector (1 if document is relevant, 0 otherwise)
        relevance = [1 if doc_id in relevant_docs[query_id] else 0 for doc_id in retrieved_doc_ids]

        # Calculate NDCG@k for this query
        ndcg_scores.append(ndcg_score([relevance], [similarities[top_k_indices]]))

    # Return the average NDCG@k score
    return sum(ndcg_scores) / len(ndcg_scores)


# Calculate and print the NDCG@10 score
ndcg_at_10 = calculate_ndcg(model=fine_tuned_model, queries=queries, corpus=corpus, relevant_docs=relevant_docs)
print(f"NDCG@10: {ndcg_at_10}")

NDCG@10: 0.884123967142861
